In [1]:
#Import libraries
import pymol
from pymol import cmd
import os

In [2]:
# Run PyMOL
pymol.finish_launching()

QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700


 PyMOL(TM) 3.0.1 - Incentive Product
 Copyright (C) Schrodinger, LLC
 
 This Executable Build integrates and extends Open-Source PyMOL.
No License File - For Evaluation Only (24 days remaining)
 Detected OpenGL version 4.1. Shaders available.
 ShaderPrg-Error: vertex shader compilation failed.; name='bezier'
 ShaderPrg-Error-InfoLog:
0:1(10): error: GLSL 4.50 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.40, 1.50, 3.30, 4.00, 4.10, 1.00 ES, and 3.00 ES

 Detected GLSL version 4.10.
 OpenGL graphics engine:
  GL_VENDOR:   Microsoft Corporation
  GL_RENDERER: D3D12 (Intel(R) UHD Graphics)
  GL_VERSION:  4.1 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
 ShaderPrg-Error: vertex shader compilation failed.; name='bezier'
 ShaderPrg-Error-InfoLog:
0:1(10): error: GLSL 4.50 is not supported. Supported versions are: 1.10, 1.20, 1.30, 1.40, 1.50, 3.30, 4.00, 4.10, 1.00 ES, and 3.00 ES


 Detected 12 CPU cores.  Enabled multithreaded rendering.


In [4]:
def load_pdb(pdb_file):
    cmd.load(pdb_file)
    #print("PDB file has been loaded.")

def select_chain(chain_name):
    cmd.select(f"all_residues_{chain_name}", f"chain {chain_name}")
    return f"all_residues_{chain_name}"   # Return the name of the new selection

def select_residues_within_distance(residue_selection, distance):
    # Define a reference selection if it's not already defined
    if not cmd.count_atoms(residue_selection):
        print(f"Error: Selection 'chain {residue_selection}' not found.")
        return
    
    # Select residues within the specified distance around the reference selection
    cmd.select(f"residues_within_{distance}_angstrom", f"byres {residue_selection} around {distance}")

    return f"residues_within_{distance}_angstrom"   # Return the name of the new selection

def code_three_to_one(aa_code_3):
    # Dictionary mapping capitalized 3-letter codes to 1-letter codes
    aa_dict = {
        'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
        'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
        'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
        'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
        'SEC': 'U', 'PYL': 'O', 'ASX': 'B', 'GLX': 'Z', 'XAA': 'X'
    }
    
    return aa_dict[aa_code_3]

def write_residues(residues_selection, chain_name, output_file):
    # Get the model of the newly selected residues
    selected_model = cmd.get_model(residues_selection)
    
    # Keep track of unique residue IDs
    unique_residues = set()
    
    # Write the amino acids and positions to the output file
    with open(output_file, "w") as f:
        for atom in selected_model.atom:    # Can only loop through atoms
            if atom.resn and atom.resi:     # Check if the atom has a residue name and number
                residue_id = (atom.resn, atom.resi)
                if residue_id not in unique_residues:
                    f.write(f"{code_three_to_one(atom.resn)}{chain_name}{atom.resi}\n")
                    unique_residues.add(residue_id)
    #print(f"Residues within {distance} Ångstroms of the reference selection have been written to {output_file}.")

In [5]:
# Specify the path to the PDB file
pdb_path = "../data/scan_complex_pdbs/Q8NBB4_P28698_scand_complex.pdb"

# Load the PDB file
load_pdb(pdb_path)

# Select chain B
chain_B_selection = select_chain("B")

# Get interactor residues for chain B (residues in chain A within 6 Ångstroms of chain B)
interactor_residues = select_residues_within_distance(chain_B_selection, 6)

# Write the interactor residues to a file
write_residues(interactor_residues, "A", "../data/interaction_residues/Q8NBB4_P28698_scand_complex_interactor_residues.txt")

61


In [9]:
def select_residues_within_distance(reference_selection, distance, output_file):
    # Define a reference selection if it's not already defined
    if not cmd.count_atoms(reference_selection):
        #print(f"Error: Selection '{reference_selection}' not found.")
        return
    
    # Select residues within the specified distance around the reference selection
    cmd.select(f"residues_within_{distance}_angstrom", f"byres {reference_selection} around {distance}")

    return f"residues_within_{distance}_angstrom"   # Return the name of the new selection

def write_residues(residues_selection, output_file):
    # Get the model of the newly selected residues
    selected_model = cmd.get_model(residues_selection)
    
    # Keep track of unique residue IDs
    unique_residues = set()
    
    # Write the amino acids and positions to the output file
    with open(output_file, "w") as f:
        for atom in selected_model.atom:    # Can only loop through atoms
            if atom.resn and atom.resi:     # Check if the atom has a residue name and number
                print(atom.resn, atom.resi)
                residue_id = (atom.resn, atom.resi)
                if residue_id not in unique_residues:
                    #f.write(f"{atom.resn} {atom.resi}\n")
                    unique_residues.add(residue_id)
                    print(residue_id)
    
    #print(f"Residues within {distance} Ångstroms of the reference selection have been written to {output_file}.")

In [5]:
def fetch_and_select_residues(protein_id, chain, output_file):
    # Fetch the protein
    cmd.fetch(protein_id)
    cmd.remove("solvent")
    
    # Select the specified chain
    cmd.select(chain, f"chain {chain}")
    
    # Perform the same operations as for protein1_id on Chain B
    select_all_residues(chain)
    select_residues_within_distance(f"{chain}_", 6, output_file)  # Update this line

In [6]:
def load_and_select_chains(pdb_path, chain_A, chain_B, output_file_prefix):
    # Load PDB file
    cmd.load(pdb_path)
    
    # Select Chain A and Chain B
    cmd.select("chain_A", f"chain {chain_A}")
    cmd.select("chain_B", f"chain {chain_B}")
    
    # Construct output file name for Chain B
    output_file_B = f"{output_file_prefix}_{chain_B}_residues.txt"
    
    # Fetch and select residues for Chain B
    fetch_and_select_residues("SCAN_domain_P28698_model0_checked_Repair", chain_B, output_file_B)

In [7]:
# Specify the path to the PDB file
pdb_path = "../data/scan_complex_pdbs/Q8NBB4_P28698_scand_complex.pdb"

# Define the chain IDs
chain_A = "A"
chain_B = "B"

# Extract the unique identifier from the PDB file name
output_file_prefix = os.path.splitext(os.path.basename(pdb_path))[0]

# Load and select residues for Chain B
load_and_select_chains(pdb_path, chain_A, chain_B, output_file_prefix)

 Error-fetch: unable to load 'scan_domain_p28698_model0_checked_repair'.
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
ARG 3
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
PHE 4
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
ARG 5
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
CYS 6
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
PHE 7
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
ARG 8
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
TYR 9
GLU 10
GLU 10
GLU 10
GLU 10
GLU 10
GLU 10
GLU 10
GLU 10
GLU 10


In [8]:
cmd.delete("all")